<a href="https://colab.research.google.com/github/SashaHub/mace_prediction/blob/main/MedGemma_adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Imports and installations

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [ ]:
!pip install xmltodict
!pip install transformers datasets accelerate peft bitsandbytes sentencepiece
!pip install --upgrade --no-cache-dir --no-deps unsloth

In [ ]:
import os
import glob
import xmltodict
import pandas as pd
import numpy as np
import re
from datetime import datetime
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    BitsAndBytesConfig # Import BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
import json # Для отладки и сохранения промежуточных данных, если нужно

# --- Константы и определения ---

# Коды МКБ-10 для MACE (можно расширить при необходимости)
# Инфаркт миокарда (острый и повторный)
MI_CODES = [f"I21.{i}" for i in range(10)] + \
           [f"I21.{i}{j}" for i in range(10) for j in range(10)] + \
           [f"I22.{i}" for i in range(10)] + \
           [f"I22.{i}{j}" for i in range(10) for j in range(10)]
# Инсульты
STROKE_CODES = [f"I60.{i}" for i in range(10)] + \
               [f"I60.{i}{j}" for i in range(10) for j in range(10)] + \
               [f"I61.{i}" for i in range(10)] + \
               [f"I61.{i}{j}" for i in range(10) for j in range(10)] + \
               [f"I62.{i}" for i in range(10)] + \
               [f"I62.{i}{j}" for i in range(10) for j in range(10)] + \
               [f"I63.{i}" for i in range(10)] + \
               [f"I63.{i}{j}" for i in range(10) for j in range(10)] + \
               [f"I64"] + \
               [f"I64.{i}" for i in range(10)] + \
               [f"I64.{i}{j}" for i in range(10) for j in range(10)] + \
               [f"I65.{i}" for i in range(10)] + \
               [f"I65.{i}{j}" for i in range(10) for j in range(10)] + \
               [f"I66.{i}" for i in range(10)] + \
               [f"I66.{i}{j}" for i in range(10) for j in range(10)] + \
               [f"I67.{i}" for i in range(10)] + \
               [f"I67.{i}{j}" for i in range(10) for j in range(10)] + \
               [f"I68.{i}" for i in range(10)] + \
               [f"I68.{i}{j}" for i in range(10) for j in range(10)] + \
               [f"I69.{i}" for i in range(10)] + \
               [f"I69.{i}{j}" for i in range(10) for j in range(10)]

MACE_ICD_CODES = set(MI_CODES + STROKE_CODES)

# Возможные отображения смерти в поле исхода госпитализации (может потребоваться дополнение)
DEATH_DISPLAY_NAMES = ["смерть", "умер", "умерла", "летальный исход"]

# Общий начальный путь для многих секций в структуре CDA
# ['ClinicalDocument', 'component', 'structuredBody', 'component', 'section']
BASE_PATH_STRUCTURED_BODY_COMP_SECTION = ['ClinicalDocument', 'component', 'structuredBody', 'component']

# Functions

In [ ]:
def find_section_by_path(data_dict, path, default=None):
    """
    Безопасно извлекает значение из вложенного словаря по списку ключей/индексов.
    Args:
        data_dict (dict): Словарь с данными документа.
        path (list): Список ключей и/или индексов для навигации.
        default: Значение по умолчанию, если путь не найден.
    Returns:
        Извлеченное значение или default.
    """
    current = data_dict
    try:
        for key_or_index in path:
            if isinstance(current, list):
                current = current[key_or_index]
            else:
                current = current[key_or_index]
        return current
    except (KeyError, IndexError, TypeError):
        return default

def clean_text(text):
    """
    Очищает текст от лишних символов, множественных пробелов, заменяет плейсхолдеры.
    Сохраняет цифры, точки, запятые, дефисы, русские и латинские буквы, знаки процента, скобки, слеши.
    Args:
        text (str): Входная строка.
    Returns:
        str: Очищенная строка.
    """
    if not isinstance(text, str):
        return ""
    text = text.replace('!!!!!!!!!!!!!', ' ')
    text = text.replace('<.>', ' ')
    text = re.sub(r'[^\w\s\.,/\-\%()\<\>]', ' ', text, flags=re.UNICODE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def parse_xml_file(file_path):
    """
    Читает XML файл и конвертирует его в словарь Python.
    Args:
        file_path (str): Путь к XML файлу.
    Returns:
        dict: Словарь, представляющий XML, или None при ошибке.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            xml_content = f.read()
        xml_content = re.sub(r'<\?xml-stylesheet.*?\?>', '', xml_content)
        data_dict = xmltodict.parse(xml_content)
        return data_dict
    except Exception as e:
        print(f"Ошибка парсинга XML файла {file_path}: {e}")
        return None

def extract_table_text_from_html_like(text_node):
    """
    Извлекает текстовое содержимое из HTML-подобной таблицы внутри XML-узла <text>.
    Args:
        text_node: Узел <text> из XML, преобразованный в словарь.
    Returns:
        str: Конкатенированный текст из ячеек таблицы.
    """
    if not text_node or not isinstance(text_node, dict):
        return ""
    all_text_parts = []
    def recurse_extract(element):
        if isinstance(element, str):
            cleaned = clean_text(element)
            if cleaned: all_text_parts.append(cleaned)
        elif isinstance(element, dict):
            if '#text' in element:
                cleaned = clean_text(element['#text'])
                if cleaned: all_text_parts.append(cleaned)
            for key in element:
                if key != '#text':
                    recurse_extract(element[key])
        elif isinstance(element, list):
            for item in element:
                recurse_extract(item)
    recurse_extract(text_node)
    return " ".join(all_text_parts)

In [ ]:
def get_patient_id(doc_dict):
    """
    Извлекает идентификатор пациента из структуры документа CDA.
    Путь: ClinicalDocument -> recordTarget -> patientRole -> id (первый элемент списка) -> @extension.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: ID пациента или None, если идентификатор не найден по указанному пути.
    """
    return find_section_by_path(doc_dict, ['ClinicalDocument', 'recordTarget', 'patientRole', 'id', 0, '@extension'])

def get_patient_sex(doc_dict):
    """
    Извлекает пол пациента из структуры документа CDA.
    Путь: ClinicalDocument -> recordTarget -> patientRole -> patient -> administrativeGenderCode -> @displayName.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Пол пациента (например, "Мужской", "Женский") или None, если информация о поле не найдена.
    """
    return find_section_by_path(doc_dict, ['ClinicalDocument', 'recordTarget', 'patientRole', 'patient', 'administrativeGenderCode', '@displayName'])

def get_patient_birth_date_str(doc_dict):
    """
    Извлекает дату рождения пациента в виде строки из структуры документа CDA.
    Путь: ClinicalDocument -> recordTarget -> patientRole -> patient -> birthTime -> @value.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Дата рождения в формате "YYYYMMDD..." или None, если дата рождения не найдена.
    """
    return find_section_by_path(doc_dict, ['ClinicalDocument', 'recordTarget', 'patientRole', 'patient', 'birthTime', '@value'])

def get_admission_date_str(doc_dict):
    """
    Извлекает дату поступления в стационар в виде строки из структуры документа CDA.
    Проверяет два возможных пути, так как дата может находиться в разных секциях:
    1. ClinicalDocument -> documentationOf -> serviceEvent -> effectiveTime -> low -> @value
    2. ClinicalDocument -> componentOf -> encompassingEncounter -> effectiveTime -> low -> @value
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Дата поступления в формате "YYYYMMDD..." или None, если дата не найдена по обоим путям.
    """
    date_val = find_section_by_path(doc_dict, ['ClinicalDocument', 'documentationOf', 'serviceEvent', 'effectiveTime', 'low', '@value'])
    if date_val:
        return date_val
    return find_section_by_path(doc_dict, ['ClinicalDocument', 'componentOf', 'encompassingEncounter', 'effectiveTime', 'low', '@value'])

def get_discharge_date_str(doc_dict):
    """
    Извлекает дату выписки из стационара в виде строки из структуры документа CDA.
    Проверяет два возможных пути, аналогично дате поступления, но использует 'high' вместо 'low':
    1. ClinicalDocument -> documentationOf -> serviceEvent -> effectiveTime -> high -> @value
    2. ClinicalDocument -> componentOf -> encompassingEncounter -> effectiveTime -> high -> @value
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Дата выписки в формате "YYYYMMDD..." или None, если дата не найдена по обоим путям.
    """
    date_val = find_section_by_path(doc_dict, ['ClinicalDocument', 'documentationOf', 'serviceEvent', 'effectiveTime', 'high', '@value'])
    if date_val:
        return date_val
    return find_section_by_path(doc_dict, ['ClinicalDocument', 'componentOf', 'encompassingEncounter', 'effectiveTime', 'high', '@value'])

def get_age_at_admission(birth_date_str, admission_date_str):
    """
    Вычисляет возраст пациента в полных годах на момент поступления.
    Для вычисления используются только год, месяц и день из предоставленных строк дат.
    Args:
        birth_date_str (str): Строка с датой рождения в формате "YYYYMMDD..." (или None).
        admission_date_str (str): Строка с датой поступления в формате "YYYYMMDD..." (или None).
    Returns:
        int: Возраст в полных годах или None, если одна из дат отсутствует или имеет некорректный формат.
    """
    if not birth_date_str or not admission_date_str:
        return None
    try:
        # Парсим только первые 8 символов (YYYYMMDD)
        birth_date = datetime.strptime(birth_date_str[:8], "%Y%m%d")
        admission_date = datetime.strptime(admission_date_str[:8], "%Y%m%d")
        # Вычисляем возраст: разница лет минус 1, если день рождения в текущем году еще не наступил
        age = admission_date.year - birth_date.year - ((admission_date.month, admission_date.day) < (birth_date.month, birth_date.day))
        return age
    except ValueError:
        # Возвращаем None, если формат даты некорректен для парсинга
        return None

def find_section_by_display_name(doc_dict, base_path_list, target_display_name):
    """
    Находит и извлекает текстовое содержимое секции по ее @displayName в структуре документа CDA.
    Рекурсивно ищет на двух уровнях вложенности секций, начиная с указанного базового пути.
    Пытается извлечь текст из '#text', напрямую из строки или из HTML-подобной структуры (таблицы).
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
        base_path_list (list): Список ключей/индексов, указывающий базовый путь к компонентам,
                               которые могут содержать искомые секции (например, путь к structuredBody/component).
        target_display_name (str): Значение атрибута @displayName искомой секции.
    Returns:
        str: Очищенное текстовое содержимое найденной секции или пустая строка, если секция не найдена
             или не содержит извлекаемого текста.
    """
    # Извлекаем основные компоненты, которые могут содержать секции, используя безопасный поиск
    main_components = find_section_by_path(doc_dict, base_path_list, [])
    # Убеждаемся, что main_components является списком для удобства итерации
    if not isinstance(main_components, list):
        main_components = [main_components]

    # Обходим компоненты первого уровня вложенности
    for comp_level1 in main_components:
        # Проверяем, что компонент существует и содержит ключ 'section'
        if not comp_level1 or 'section' not in comp_level1: continue
        section_level1 = comp_level1['section']

        sections_to_check_lvl1 = []
        # Преобразуем section_level1 в список, если это не список (для случая одной секции)
        if isinstance(section_level1, list): sections_to_check_lvl1.extend(section_level1)
        elif isinstance(section_level1, dict): sections_to_check_lvl1.append(section_level1)

        # Обходим секции первого уровня
        for sec_l1_dict in sections_to_check_lvl1:
            if not sec_l1_dict: continue
            # Проверяем, совпадает ли @displayName текущей секции с целевым
            if sec_l1_dict.get('code', {}).get('@displayName') == target_display_name:
                # Если нашли секцию, пытаемся извлечь ее текстовое содержимое
                text_node = sec_l1_dict.get('text')
                # Проверяем разные возможные места хранения текста: '#text', прямой текст, entry/observation/value, HTML-подобная таблица
                if isinstance(text_node, dict) and '#text' in text_node: return clean_text(text_node['#text'])
                if isinstance(text_node, str): return clean_text(text_node)
                entry_val = find_section_by_path(sec_l1_dict, ['entry', 'observation', 'value'])
                if isinstance(entry_val, dict) and '#text' in entry_val: return clean_text(entry_val['#text'])
                if isinstance(entry_val, str): return clean_text(entry_val)
                content_val = find_section_by_path(text_node, ['content'])
                if content_val: return extract_table_text_from_html_like(text_node)

            # Если секция первого уровня не целевая, проверяем ее вложенные компоненты (второй уровень)
            components_level2 = find_section_by_path(sec_l1_dict, ['component'], [])
            # Убеждаемся, что components_level2 является списком
            if not isinstance(components_level2, list):
                components_level2 = [components_level2]

            # Обходим компоненты второго уровня
            for comp_level2 in components_level2:
                # Проверяем, что компонент существует и содержит ключ 'section'
                if not comp_level2 or 'section' not in comp_level2: continue
                section_level2 = comp_level2['section']
                sections_to_check_lvl2 = []
                # Преобразуем section_level2 в список
                if isinstance(section_level2, list): sections_to_check_lvl2.extend(section_level2)
                elif isinstance(section_level2, dict): sections_to_check_lvl2.append(section_level2)

                # Обходим секции второго уровня
                for sec_l2_dict in sections_to_check_lvl2:
                    if not sec_l2_dict: continue
                    # Проверяем, совпадает ли @displayName текущей секции с целевым
                    if sec_l2_dict.get('code', {}).get('@displayName') == target_display_name:
                        # Если нашли секцию, пытаемся извлечь ее текстовое содержимое (аналогично первому уровню)
                        text_node = sec_l2_dict.get('text')
                        if isinstance(text_node, dict) and '#text' in text_node: return clean_text(text_node['#text'])
                        if isinstance(text_node, str): return clean_text(text_node)
                        entry_val = find_section_by_path(sec_l2_dict, ['entry', 'observation', 'value'])
                        if isinstance(entry_val, dict) and '#text' in entry_val: return clean_text(entry_val['#text'])
                        if isinstance(entry_val, str): return clean_text(entry_val)
                        content_val = find_section_by_path(text_node, ['content'])
                        if content_val: return extract_table_text_from_html_like(text_node)
    return "" # Возвращаем пустую строку, если секция с целевым @displayName не найдена на обоих уровнях.

def get_anamnesis_disease(doc_dict):
    """
    Извлекает текстовое содержимое секции "Анамнез заболевания" по ее @displayName.
    Использует find_section_by_display_name с базовым путем к структурированному телу документа.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Очищенный текст анамнеза заболевания или пустая строка, если секция не найдена.
    """
    return find_section_by_display_name(doc_dict, BASE_PATH_STRUCTURED_BODY_COMP_SECTION, "Анамнез заболевания")

def get_anamnesis_life(doc_dict):
    """
    Извлекает текстовое содержимое секции "Анамнез жизни" по ее @displayName.
    Использует find_section_by_display_name с базовым путем к структурированному телу документа.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Очищенный текст анамнеза жизни или пустая строка, если секция не найдена.
    """
    return find_section_by_display_name(doc_dict, BASE_PATH_STRUCTURED_BODY_COMP_SECTION, "Анамнез жизни")

def get_condition_on_admission(doc_dict):
    """
    Извлекает текстовое содержимое секции "Состояние при поступлении" по ее @displayName.
    Использует find_section_by_display_name с базовым путем к структурированному телу документа.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Очищенный текст описания состояния при поступлении или пустая строка, если секция не найдена.
    """
    return find_section_by_display_name(doc_dict, BASE_PATH_STRUCTURED_BODY_COMP_SECTION, "Состояние при поступлении")

def get_diagnoses_from_section(section_dict, section_title_target):
    """
    Извлекает список диагнозов (код МКБ и текст описания) из заданной секции документа CDA.
    Функция ищет вложенные секции с @displayName="Диагнозы" и проверяет их заголовок на соответствие section_title_target.
    Диагнозы извлекаются как из структуры <entry>/<observation>, так и из HTML-подобных таблиц в <text>.
    Args:
        section_dict (dict): Словарь, представляющий родительскую секцию, которая может содержать вложенные секции "Диагнозы".
        section_title_target (str): Ожидаемое начало текста заголовка вложенной секции "Диагнозы"
                                    (например, "Установленные диагнозы при поступлении").
    Returns:
        list: Список словарей, каждый из которых представляет диагноз: {'mkb_code': '...', 'text': '...'}.
              Возвращает пустой список, если диагнозы не найдены.
    """
    diagnoses = []
    # Ищем вложенные компоненты внутри родительской секции
    components = find_section_by_path(section_dict, ['component'], [])
    # Убеждаемся, что components является списком
    if not isinstance(components, list): components = [components]

    # Обходим вложенные компоненты
    for comp in components:
        diag_section = comp.get('section', {})
        # Проверяем, является ли вложенная секция секцией диагнозов с нужным @displayName и заголовком
        if diag_section.get('code', {}).get('@displayName') == "Диагнозы" and \
           clean_text(find_section_by_path(diag_section,['title','#text'],"")).startswith(section_title_target):

            # --- Извлечение диагнозов из структуры <entry>/<observation> ---
            entries = find_section_by_path(diag_section, ['entry'], [])
            # Убеждаемся, что entries является списком
            if not isinstance(entries, list): entries = [entries]
            for entry_item in entries:
                # Пропускаем записи с определенным codeSystem (часто служебные или недиагнозы)
                if find_section_by_path(entry_item, ['act', 'code', '@codeSystem']) == "1.2.643.5.1.13.13.99.2.795":
                    continue
                # Диагноз может быть вложен в entryRelationship/observation
                obs_list = find_section_by_path(entry_item, ['observation', 'entryRelationship', 'observation'], [])
                # Убеждаемся, что obs_list является списком
                if not isinstance(obs_list, list): obs_list = [obs_list]
                for obs in obs_list:
                    if not obs: continue
                    # Извлекаем код МКБ (@code) и текст диагноза (#text или @displayName)
                    icd_code = find_section_by_path(obs, ['value', '@code'])
                    diag_text = clean_text(find_section_by_path(obs, ['text', '#text']))
                    if not diag_text: # Если текст не найден в <text>, ищем в @displayName value
                        diag_text = clean_text(find_section_by_path(obs, ['value', '@displayName']))
                    if icd_code and diag_text:
                        diagnoses.append({'mkb_code': icd_code.strip(), 'text': diag_text.strip()})

            # --- Извлечение диагнозов из HTML-подобной таблицы в <text> ---
            text_node = diag_section.get('text')
            if text_node and 'table' in text_node:
                table_data = text_node['table']
                tbody = table_data.get('tbody')
                if tbody and 'tr' in tbody:
                    rows = tbody['tr']
                    # Убеждаемся, что rows является списком строк
                    if not isinstance(rows, list): rows = [rows]
                    for row in rows:
                        cols = row.get('td') # Ячейки таблицы
                        # Проверяем, что это список ячеек и их достаточно (номер, описание, код)
                        if isinstance(cols, list) and len(cols) >= 3:
                            # Извлекаем описание и код из соответствующих ячеек, проверяя разные пути к тексту
                            col1_content = find_section_by_path(cols[1], ['content', '#text']) or find_section_by_path(cols[1],['content']) or find_section_by_path(cols[1],['#text'])
                            col2_content = find_section_by_path(cols[2], ['content', '#text']) or find_section_by_path(cols[2],['content']) or find_section_by_path(cols[2],['#text'])
                            desc_text = clean_text(col1_content)
                            code_text = clean_text(col2_content)
                            if code_text and desc_text:
                                 diagnoses.append({'mkb_code': code_text.strip(), 'text': desc_text.strip()})
    return diagnoses

def get_all_diagnoses(doc_dict, admission_or_discharge="admission"):
    """
    Извлекает все диагнозы (при поступлении или при выписке) из документа CDA.
    Функция ищет диагнозы в различных стандартных секциях ("Состояние при поступлении",
    "Пребывание в стационаре", "Состояние при выписке") и их вложенных секциях "Диагнозы".
    Также включает логику извлечения диагнозов из таблиц в секции "Пребывание в стационаре".
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
        admission_or_discharge (str): Указывает, какие диагнозы искать: "admission" для диагнозов при поступлении,
                                      "discharge" для диагнозов при выписке.
    Returns:
        list: Список уникальных словарей {'mkb_code': '...', 'text': '...'} найденных диагнозов.
              Уникальность определяется по коду МКБ (без подкатегории) и тексту диагноза.
              Возвращает пустой список, если диагнозы не найдены.
    """
    all_diagnoses_list = []
    # Ищем контейнеры секций в структурированном теле документа
    section_containers = find_section_by_path(doc_dict, BASE_PATH_STRUCTURED_BODY_COMP_SECTION, [])
    # Убеждаемся, что section_containers является списком
    if not isinstance(section_containers, list): section_containers = [section_containers]

    # Определяем целевые названия родительских секций и заголовков вложенных секций "Диагнозы"
    # в зависимости от того, ищем ли мы диагнозы при поступлении или при выписке.
    target_parent_section_name = "Пребывание в стационаре" if admission_or_discharge == "admission" else "Состояние при выписке"
    target_diag_section_title = "Установленные диагнозы при поступлении" if admission_or_discharge == "admission" else "Установленные диагнозы при выписке"

    # Обходим контейнеры секций
    for section_container_item in section_containers:
        if not section_container_item or 'section' not in section_container_item: continue # Пропускаем пустые или без секций
        current_section_candidates = section_container_item['section']
        # Убеждаемся, что current_section_candidates является списком
        if not isinstance(current_section_candidates, list): current_section_candidates = [current_section_candidates]

        # Обходим секции первого уровня в контейнере
        for current_section_dict in current_section_candidates:
            if not current_section_dict: continue
            # Получаем @displayName текущей секции
            parent_display_name = current_section_dict.get('code', {}).get('@displayName')

            # Если это секция "Состояние при поступлении" и мы ищем диагнозы при поступлении
            if parent_display_name == "Состояние при поступлении" and admission_or_discharge == "admission":
                 # Извлекаем диагнозы из этой секции, используя целевой заголовок
                 diagnoses = get_diagnoses_from_section(current_section_dict, target_diag_section_title)
                 all_diagnoses_list.extend(diagnoses)
            # Если это целевая родительская секция ("Пребывание в стационаре" или "Состояние при выписке")
            elif parent_display_name == target_parent_section_name:
                # Извлекаем диагнозы из этой секции, используя целевой заголовок
                diagnoses = get_diagnoses_from_section(current_section_dict, target_diag_section_title)
                all_diagnoses_list.extend(diagnoses)
            # Специальная логика для извлечения диагнозов из таблицы в секции "Пребывание в стационаре"
            # (часто там указываются основные диагнозы пребывания)
            if admission_or_discharge == "admission" and parent_display_name == "Пребывание в стационаре":
                text_node_main = current_section_dict.get('text')
                if text_node_main and 'table' in text_node_main:
                    # Проверяем заголовки таблицы, чтобы убедиться, что это таблица диагнозов
                    headers_text = extract_table_text_from_html_like(text_node_main.get('table',{}).get('thead',{}))
                    if "Вид нозологической единицы" in headers_text and "Код по МКБ-10" in headers_text:
                        table_data = text_node_main['table']
                        tbody = table_data.get('tbody')
                        if tbody and 'tr' in tbody:
                            rows = tbody['tr']
                            # Убеждаемся, что rows является списком строк
                            if not isinstance(rows, list): rows = [rows]
                            for row in rows:
                                cols = row.get('td') # Ячейки таблицы
                                # Проверяем, что это список ячеек и их достаточно
                                if isinstance(cols, list) and len(cols) >= 3:
                                    # Извлекаем описание и код из соответствующих ячеек
                                    desc_node_val = find_section_by_path(cols[1], ['content', '#text']) or find_section_by_path(cols[1],['content']) or find_section_by_path(cols[1],['#text'])
                                    code_node_val = find_section_by_path(cols[2], ['content', '#text']) or find_section_by_path(cols[2],['content']) or find_section_by_path(cols[2],['#text'])
                                    desc_text = clean_text(desc_node_val)
                                    code_text = clean_text(code_node_val)
                                    if code_text and desc_text:
                                        all_diagnoses_list.append({'mkb_code': code_text, 'text': desc_text})

    # Удаляем дубликаты диагнозов. Уникальность определяется по коду МКБ (берем только основную часть) и тексту.
    unique_diagnoses = []
    seen_tuples = set()
    for d in all_diagnoses_list:
        mkb_code_cleaned = d['mkb_code'].split()[0] if d['mkb_code'] else "" # Берем только часть кода до первого пробела (например, "I21.4" из "I21.4 MACE")
        if mkb_code_cleaned and (mkb_code_cleaned, d['text']) not in seen_tuples:
            unique_diagnoses.append({'mkb_code': mkb_code_cleaned, 'text': d['text']})
            seen_tuples.add((mkb_code_cleaned, d['text']))
    return unique_diagnoses

def get_discharge_outcome(doc_dict):
    """
    Извлекает исход госпитализации из структуры документа CDA.
    Путь: ClinicalDocument -> componentOf -> encompassingEncounter -> dischargeDispositionCode -> @displayName.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Описание исхода госпитализации (@displayName, например "выписан", "смерть")
             или None, если исход не найден.
    """
    return find_section_by_path(doc_dict, ['ClinicalDocument', 'componentOf', 'encompassingEncounter', 'dischargeDispositionCode', '@displayName'])

def get_instrumental_studies_text(doc_dict):
    """
    Извлекает текстовое содержимое секций с инструментальными исследованиями из документа CDA.
    Функция ищет секции внутри структуры 'PATIENTROUTE' (маршрут пациента) и 'PROC' (процедуры).
    Собирает названия отделений, исследований, текст протоколов/результатов (включая таблицы)
    и детали исследований из entry/observation.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Объединенный текст из всех найденных секций исследований, очищенный.
             Возвращает пустую строку, если секции исследований не найдены.
    """
    studies_texts = []
    # Ищем контейнеры секций в структурированном теле документа
    section_containers = find_section_by_path(doc_dict, BASE_PATH_STRUCTURED_BODY_COMP_SECTION, [])
    # Убеждаемся, что section_containers является списком
    if not isinstance(section_containers, list): section_containers = [section_containers]

    # Обходим контейнеры секций
    for sc in section_containers:
        if not sc or 'section' not in sc: continue # Пропускаем пустые или без секций
        top_level_section = sc['section']
        sections_to_scan_for_patientroute = []
        # Преобразуем top_level_section в список
        if isinstance(top_level_section, list): sections_to_scan_for_patientroute.extend(top_level_section)
        else: sections_to_scan_for_patientroute.append(top_level_section)

        # Ищем секцию с кодом 'PATIENTROUTE' (маршрут пациента)
        for s_l1 in sections_to_scan_for_patientroute:
            if not s_l1: continue
            if find_section_by_path(s_l1, ['code', '@code']) == 'PATIENTROUTE':
                # Ищем вложенные компоненты (обычно отделения или этапы маршрута)
                departments_components = find_section_by_path(s_l1, ['component'], [])
                # Убеждаемся, что departments_components является списком
                if not isinstance(departments_components, list): departments_components = [departments_components]
                for dept_comp in departments_components:
                    dept_section = dept_comp.get('section')
                    if not dept_section: continue
                    # Извлекаем название отделения/этапа
                    dept_name = clean_text(find_section_by_path(dept_section, ['title', '#text']))
                    if dept_name: studies_texts.append(f"Отделение: {dept_name}")

                    # Ищем вложенные компоненты с секциями процедур ('PROC')
                    researches_main_component = find_section_by_path(dept_section, ['component'], [])
                    # Убеждаемся, что researches_main_component является списком
                    if not isinstance(researches_main_component, list): researches_main_component = [researches_main_component]
                    for res_main_comp_item in researches_main_component:
                        proc_section = res_main_comp_item.get('section')
                        # Проверяем, что это секция процедур
                        if not proc_section or find_section_by_path(proc_section,['code','@code']) != 'PROC':
                            continue
                        # Ищем вложенные компоненты с деталями исследований
                        actual_researches_components = find_section_by_path(proc_section, ['component'],[])
                        # Убеждаемся, что actual_researches_components является списком
                        if not isinstance(actual_researches_components, list): actual_researches_components = [actual_researches_components]
                        for actual_res_comp in actual_researches_components:
                            research_detail_section = actual_res_comp.get('section')
                            if not research_detail_section: continue
                            # Извлекаем название исследования
                            research_name = clean_text(find_section_by_path(research_detail_section, ['title', '#text']))
                            if research_name: studies_texts.append(f"Исследование ({dept_name if dept_name else 'Общее'}): {research_name}")

                            # Извлекаем текст протокола/результатов из <text> (часто в виде таблиц)
                            text_node = research_detail_section.get('text')
                            table_text = extract_table_text_from_html_like(text_node)
                            if table_text: studies_texts.append(f"Протокол/Результаты: {table_text}")

                            # Извлекаем детали исследования из entry/observation
                            entries = find_section_by_path(research_detail_section, ['entry'], [])
                            # Убеждаемся, что entries является списком
                            if not isinstance(entries, list): entries = [entries]
                            for entry in entries:
                                # Извлекаем название параметра (@displayName code) и его значение (#text value или @displayName value)
                                obs_code_dn = clean_text(find_section_by_path(entry, ['observation', 'code', '@displayName']))
                                obs_value_text = clean_text(find_section_by_path(entry, ['observation', 'value', '#text']))
                                obs_value_dn = clean_text(find_section_by_path(entry, ['observation', 'value', '@displayName']))
                                if obs_code_dn and obs_value_text:
                                    studies_texts.append(f"{obs_code_dn}: {obs_value_text}")
                                elif obs_code_dn and obs_value_dn and obs_value_dn != obs_code_dn : # Добавляем значение, если оно отличается от названия параметра
                                    studies_texts.append(f"{obs_code_dn}: {obs_value_dn}")
                                elif obs_value_text: # Если нет названия параметра, добавляем только текстовое значение
                                    studies_texts.append(f"Деталь исследования: {obs_value_text}")

    # Объединяем все собранные текстовые части исследований в одну строку, удаляя пустые
    return " ".join(filter(None, studies_texts))

def get_general_hospitalization_info_text(doc_dict):
    """
    Извлекает текстовое содержимое секции с общими сведениями о госпитализации ('HOSP').
    Ищет секцию по ее коду 'HOSP' на первом уровне вложенности в структурированном теле.
    Извлекает текст из HTML-подобной структуры внутри <text>.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Очищенный текст общих сведений о госпитализации или пустая строка, если секция не найдена.
    """
    section_containers = find_section_by_path(doc_dict, BASE_PATH_STRUCTURED_BODY_COMP_SECTION, [])
    # Убеждаемся, что section_containers является списком
    if not isinstance(section_containers, list): section_containers = [section_containers]
    # Обходим контейнеры секций
    for sc in section_containers:
        if not sc or 'section' not in sc: continue # Пропускаем пустые или без секций
        hosp_section_candidates = sc['section']
        # Убеждаемся, что hosp_section_candidates является списком
        if not isinstance(hosp_section_candidates, list): hosp_section_candidates = [hosp_section_candidates]
        # Ищем секцию с кодом 'HOSP'
        for hs in hosp_section_candidates:
            if hs and find_section_by_path(hs,['code','@code']) == 'HOSP':
                 text_node = hs.get('text')
                 # Извлекаем текст из HTML-подобной структуры внутри <text>
                 return extract_table_text_from_html_like(text_node)
    return "" # Возвращаем пустую строку, если секция не найдена

def get_discharge_condition_text(doc_dict):
    """
    Извлекает текстовое содержимое секции "Состояние при выписке".
    Ищет секцию как по @displayName="Состояние при выписке", так и по коду 'STATEDIS'.
    По коду 'STATEDIS' ищет на первом уровне вложенности и вложенную в секцию 'HOSP'.
    Извлекает текст из HTML-подобной структуры внутри <text>.
    Args:
        doc_dict (dict): Словарь, представляющий XML документа CDA.
    Returns:
        str: Очищенный текст описания состояния при выписке или пустая строка, если секция не найдена.
    """
    # Сначала пробуем найти по display name (менее надежно, но может сработать)
    text_content = find_section_by_display_name(doc_dict, BASE_PATH_STRUCTURED_BODY_COMP_SECTION, "Состояние при выписке")

    section_containers = find_section_by_path(doc_dict, BASE_PATH_STRUCTURED_BODY_COMP_SECTION, [])
    # Убеждаемся, что section_containers является списком
    if not isinstance(section_containers, list): section_containers = [section_containers]

    # Обходим контейнеры секций для поиска по коду 'STATEDIS'
    for sc in section_containers:
        if not sc or 'section' not in sc: continue # Пропускаем пустые или без секций
        current_level1_sections = sc['section']
        # Убеждаемся, что current_level1_sections является списком
        if not isinstance(current_level1_sections, list): current_level1_sections = [current_level1_sections]

        # Ищем секцию с кодом 'STATEDIS' на первом уровне
        for sec_l1 in current_level1_sections:
            if sec_l1 and find_section_by_path(sec_l1,['code','@code']) == 'STATEDIS':
                 text_node = sec_l1.get('text')
                 # Извлекаем текст из HTML-подобной структуры внутри <text>
                 return extract_table_text_from_html_like(text_node)
            # Ищем секцию с кодом 'STATEDIS' вложенную в секцию 'HOSP'
            if sec_l1 and find_section_by_path(sec_l1,['code','@code']) == 'HOSP':
                components_l2 = find_section_by_path(sec_l1, ['component'], [])
                # Убеждаемся, что components_l2 является списком
                if not isinstance(components_l2, list): components_l2 = [components_l2]
                for comp_l2 in components_l2:
                    sec_l2 = comp_l2.get('section', {})
                    if sec_l2 and find_section_by_path(sec_l2,['code','@code']) == 'STATEDIS':
                        text_node = sec_l2.get('text')
                        # Извлекаем текст из HTML-подобной структуры внутри <text>
                        return extract_table_text_from_html_like(text_node)
    # Если по коду не найдено, возвращаем результат поиска по display name (может быть пустой строкой)
    return text_content

In [ ]:
def process_patient_records(patient_records_with_meta):
    if not patient_records_with_meta:
        return None

    patient_records_data = [record['data'] for record in patient_records_with_meta]
    first_visit_data = patient_records_data[0]

    patient_id = get_patient_id(first_visit_data)
    sex = get_patient_sex(first_visit_data)
    birth_date_str = get_patient_birth_date_str(first_visit_data)
    admission_date_str_first = get_admission_date_str(first_visit_data)
    age = get_age_at_admission(birth_date_str, admission_date_str_first)

    anamnesis_d = get_anamnesis_disease(first_visit_data)
    anamnesis_l = get_anamnesis_life(first_visit_data)
    condition_adm = get_condition_on_admission(first_visit_data)

    diagnoses_adm_list_first_visit = get_all_diagnoses(first_visit_data, "admission")
    diagnoses_dis_list_first_visit = get_all_diagnoses(first_visit_data, "discharge")

    diagnoses_adm_texts = [f"{d['mkb_code']}: {d['text']}" for d in diagnoses_adm_list_first_visit]
    diagnoses_dis_texts_fv = [f"{d['mkb_code']}: {d['text']}" for d in diagnoses_dis_list_first_visit]

    instrumental_text = get_instrumental_studies_text(first_visit_data)
    general_hosp_text = get_general_hospitalization_info_text(first_visit_data)
    discharge_condition_text_fv = get_discharge_condition_text(first_visit_data)

    text_features_parts = [
        f"Анамнез заболевания: {anamnesis_d}" if anamnesis_d else "",
        f"Анамнез жизни: {anamnesis_l}" if anamnesis_l else "",
        f"Состояние при поступлении: {condition_adm}" if condition_adm else "",
        f"Диагнозы при поступлении: {'; '.join(diagnoses_adm_texts)}" if diagnoses_adm_texts else "",
        f"Диагнозы при выписке (первый визит): {'; '.join(diagnoses_dis_texts_fv)}" if diagnoses_dis_texts_fv else "",
        f"Результаты исследований (первый визит): {instrumental_text}" if instrumental_text else "",
        f"Общие сведения о госпитализации (первый визит): {general_hosp_text}" if general_hosp_text else "",
        f"Состояние при выписке (первый визит): {discharge_condition_text_fv}" if discharge_condition_text_fv else ""
    ]
    aggregated_text = clean_text(" ".join(filter(None, text_features_parts)))

    # Собираем МКБ коды только со второго и последующих визитов
    subsequent_visits_mkb_codes = set()
    mace_target = None # Инициализируем таргет

    if len(patient_records_data) > 1:
        mace_target = 0 # По умолчанию таргет 0, если есть последующие визиты, но нет MACE/смерти
        for subsequent_visit_idx in range(1, len(patient_records_data)):
            subsequent_visit_data = patient_records_data[subsequent_visit_idx]
            discharge_outcome = get_discharge_outcome(subsequent_visit_data)

            if discharge_outcome and clean_text(discharge_outcome).lower() in DEATH_DISPLAY_NAMES:
                mace_target = 1
                break # Найден летальный исход в последующем визите, устанавливаем таргет 1 и выходим

            subsequent_diagnoses_adm = get_all_diagnoses(subsequent_visit_data, "admission")
            subsequent_diagnoses_dis = get_all_diagnoses(subsequent_visit_data, "discharge")

            # Собираем МКБ коды при поступлении и выписке из последующих визитов
            for diag in subsequent_diagnoses_adm + subsequent_diagnoses_dis:
                 mkb_code = diag.get('mkb_code', '').strip().upper()
                 if mkb_code:
                     subsequent_visits_mkb_codes.add(mkb_code)

            # Проверяем коды МКБ из последующих визитов на наличие MACE для установки таргета
            for diag in subsequent_diagnoses_adm + subsequent_diagnoses_dis:
                mkb_code = diag.get('mkb_code', '').strip().upper()
                if mkb_code in MACE_ICD_CODES:
                    mace_target = 1
                    break # Найден MACE код во последующем визите, устанавливаем таргет и выходим

            if mace_target == 1: # Если таргет стал 1 в этом визите, завершаем проверку последующих визитов
                break

    return {
        'patient_id': patient_id,
        'text_features': aggregated_text,
        'age_at_first_admission': age,
        'sex': sex,
        'mace_target': mace_target,
        'unique_mkb_codes': list(subsequent_visits_mkb_codes) # Коды из 2+ визитов
    }

# Download data

In [ ]:
zip_path = '/content/drive/My Drive/med_records_8500.zip'
extract_path = '/medical_records'

# Создаем папку для извлечения, если она не существует
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Файл {zip_path} успешно распакован в папку {extract_path}")

In [ ]:
xml_pattern = "/medical_records/med_records_8500/*.xml" # Пример для файлов, начинающихся с EMD_EPIC_DISCHARGE_

# Если файлы в другой директории, укажите путь:
# current_directory = os.getcwd() # Получаем текущую рабочую директорию ноутбука
# xml_pattern = os.path.join(current_directory, "ваша_папка_с_xml", "*.xml")


all_records_by_patient = {}
xml_files = glob.glob(xml_pattern)

if not xml_files:
    print(f"Не найдено XML файлов по паттерну: {xml_pattern}")
    # Можно остановить выполнение, если файлы не найдены, или продолжить с пустым списком
    # raise FileNotFoundError(f"Не найдено XML файлов по паттерну: {xml_pattern}")
else:
    print(f"Найдено {len(xml_files)} XML файлов. Начинаю обработку...")

    for file_path in xml_files:
        doc_dict = parse_xml_file(file_path)
        if not doc_dict:
            continue

        patient_id = get_patient_id(doc_dict)
        admission_date_str = get_admission_date_str(doc_dict)

        if not patient_id:
            print(f"Не удалось извлечь ID пациента из файла: {file_path}")
            continue

        admission_date = None
        if admission_date_str:
            try:
                admission_date = datetime.strptime(admission_date_str[:8], "%Y%m%d")
            except ValueError:
                print(f"Некорректный формат даты поступления '{admission_date_str}' в файле: {file_path}.")
                admission_date = datetime.max # Для некорректных дат ставим максимальную, чтобы они были в конце
        else:
             print(f"Отсутствует дата поступления в файле: {file_path}.")
             admission_date = datetime.max


        if patient_id not in all_records_by_patient:
            all_records_by_patient[patient_id] = []

        all_records_by_patient[patient_id].append({
            'admission_date': admission_date,
            'data': doc_dict,
            'file_path': file_path
        })

    for patient_id in all_records_by_patient:
        all_records_by_patient[patient_id].sort(key=lambda x: x['admission_date'])

    print(f"Сгруппировано {len(all_records_by_patient)} уникальных пациентов.")

In [ ]:
final_data_list = []
processed_patients = 0
total_patients = len(all_records_by_patient)

if total_patients > 0:
    print(f"Начинаю детальную обработку {total_patients} пациентов...")
    for patient_id, records_with_meta in all_records_by_patient.items():
        processed_info = process_patient_records(records_with_meta)
        if processed_info:
            # Добавляем количество визитов в processed_info
            processed_info['num_visits'] = len(records_with_meta)
            final_data_list.append(processed_info)
        processed_patients +=1
        if processed_patients % 100 == 0 or processed_patients == total_patients:
            print(f"Обработано пациентов: {processed_patients}/{total_patients}")

    df_results = pd.DataFrame(final_data_list)

    print("\nОбновление таргета MACE на основе кодов выписки из 2+ визитов...")
    initial_mace_1_count = df_results['mace_target'].value_counts().get(1.0, 0)

    # Проходим по DataFrame и обновляем таргет
    for index, row in df_results.iterrows():
        # Обновляем только если текущий таргет не 1 (т.е. не было летального исхода)
        # и у пациента было более одного визита
        if row['mace_target'] != 1.0 and row['num_visits'] > 1 and row['unique_mkb_codes']:
            # Проверяем наличие MACE кодов в unique_mkb_codes (коды выписки 2+ визитов)
            if any(code in MACE_ICD_CODES for code in row['unique_mkb_codes']):
                df_results.loc[index, 'mace_target'] = 1.0

    updated_mace_1_count = df_results['mace_target'].value_counts().get(1.0, 0)
    print(f"Количество пациентов с mace_target=1.0 после обновления: {updated_mace_1_count} (До обновления: {initial_mace_1_count})")


    print("\nИтоговая таблица (первые 5 строк):")
    display(df_results.head())
    print(f"\nРазмер таблицы: {df_results.shape}")

    if not df_results.empty:
        print(f"\nРаспределение MACE таргета:\n{df_results['mace_target'].value_counts(dropna=False)}")
        print(f"\nРаспределение количества визитов:\n{df_results['num_visits'].value_counts().sort_index()}")
    else:
        print("\nИтоговая таблица пуста после обработки.")
else:
    print("Нет данных для обработки (список пациентов пуст).")
    df_results = pd.DataFrame()

In [ ]:
# Сбор всех уникальных МКБ кодов из DataFrame
all_unique_mkb_codes_in_data = set()
for index, row in df_results.iterrows():
    if row['unique_mkb_codes']:
        all_unique_mkb_codes_in_data.update(row['unique_mkb_codes'])

print(f"Общее количество уникальных МКБ кодов в данных: {len(all_unique_mkb_codes_in_data)}")

# Сравнение с MACE_ICD_CODES
mace_codes_in_data = all_unique_mkb_codes_in_data.intersection(MACE_ICD_CODES)
print(f"Количество МКБ кодов из списка MACE, найденных в данных: {len(mace_codes_in_data)}")

# Вывод МКБ кодов из списка MACE, которые есть в данных
if mace_codes_in_data:
    print("\nМКБ коды из списка MACE, найденные в данных:")
    print(sorted(list(mace_codes_in_data)))
else:
    print("\nВ данных не найдено МКБ кодов, соответствующих списку MACE_ICD_CODES.")

# Вывод МКБ кодов в данных, которые не входят в список MACE_ICD_CODES (первые 20 для примера)
other_codes_in_data = list(all_unique_mkb_codes_in_data - MACE_ICD_CODES)
print(f"\nКоличество других МКБ кодов в данных (не из списка MACE): {len(other_codes_in_data)}")
if other_codes_in_data:
    print("Примеры других МКБ кодов в данных (первые 20):")
    print(sorted(other_codes_in_data)[:20])

# Анализ пациентов с mace_target=0 и NaN
mace_0_patients = df_results[df_results['mace_target'] == 0.0]
mace_nan_patients = df_results[df_results['mace_target'].isna()]

print(f"\nКоличество пациентов с mace_target = 0.0: {len(mace_0_patients)}")
print(f"Количество пациентов с mace_target = NaN: {len(mace_nan_patients)}")

# Проверка наличия MACE кодов у пациентов с mace_target = 0.0
mace_0_with_mace_codes = mace_0_patients[mace_0_patients['unique_mkb_codes'].apply(lambda codes: any(code in MACE_ICD_CODES for code in codes))]
print(f"Количество пациентов с mace_target = 0.0, у которых есть МКБ коды из списка MACE: {len(mace_0_with_mace_codes)}")

# Проверка наличия MACE кодов у пациентов с mace_target = NaN
mace_nan_with_mace_codes = mace_nan_patients[mace_nan_patients['unique_mkb_codes'].apply(lambda codes: any(code in MACE_ICD_CODES for code in codes))]
print(f"Количество пациентов с mace_target = NaN, у которых есть МКБ коды из списка MACE: {len(mace_nan_with_mace_codes)}")

## Save Data

In [ ]:
# if not df_results.empty:
#     output_filename = "/content/drive/My Drive/parsed_medical_data.parquet" # Указываем путь на Google Диск и формат Parquet
#     df_results.to_parquet(output_filename, index=False) # Используем to_parquet
#     print(f"\nТаблица сохранена в {output_filename}")
# else:
#     print("\nDataFrame пуст, сохранение не выполнено.")

# LLMS

## Download again

In [ ]:
# Загрузка таблицы из Google Диска
output_filename = "/content/drive/My Drive/parsed_medical_data.parquet"
try:
    df_results = pd.read_parquet(output_filename)
    print(f"Таблица успешно загружена из {output_filename}")
    display(df_results.head())
except Exception as e:
    print(f"Ошибка при загрузке таблицы из {output_filename}: {e}")
    df_results = pd.DataFrame() # Создаем пустой DataFrame в случае ошибки

## Text data cleaning

In [ ]:
import re

def further_clean_text(text):
    """
    Дополнительная очистка текста. Сохраняет заглавные буквы в начале предложений, аббревиатуры из 2-3 заглавных букв,
    остальной текст приводит к нижнему регистру. Удаляет повторяющиеся символы и специфические паттерны.
    """
    if not isinstance(text, str):
        return ""

    # Список служебных слов/местоимений, которые часто пишутся заглавными по ошибке
    PRONOUNS_OR_FUNCTION_WORDS = {"ЖЕ", "ТО", "ЛИ", "ИЛИ", "НЕ", "НИ", "БЫ"}

    # Предварительная очистка от специфических паттернов и лишних символов
    text = text.replace('!!!!!!!!!!!!!', ' ')
    text = text.replace('<.>', ' ')
    # Заменяем группы небуквенных символов (кроме разрешенных) на пробелы
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s\.,/\-\%()<>]+', ' ', text, flags=re.UNICODE)
    # Заменяем множественные пробелы на один и удаляем пробелы по краям
    text = re.sub(r'\s+', ' ', text).strip()

    # Обработка заглавных букв и аббревиатур
    # Используем более надежное разбиение на предложения, сохраняя разделители для последующего объединения
    sentence_parts = re.split(r'([.!?]\s+)', text)
    processed_sentences = []

    current_sentence_words = []
    is_first_word_of_sentence = True

    for part in sentence_parts:
        if not part:
            continue
        if re.fullmatch(r'[.!?]\s+', part): # Если часть - это разделитель предложения
            if current_sentence_words:
                processed_sentences.append(" ".join(current_sentence_words))
            processed_sentences.append(part.strip()) # Добавляем разделитель
            current_sentence_words = [] # Начинаем новое предложение
            is_first_word_of_sentence = True
        else: # Если часть - это текст предложения
            words = part.split()
            for word in words:
                # Проверяем, является ли слово аббревиатурой (2 или 3 заглавные буквы)
                if re.fullmatch(r'[А-ЯA-Z]{2,3}', word):
                    current_sentence_words.append(word) # Сохраняем как аббревиатуру
                    is_first_word_of_sentence = False
                # Проверяем, является ли слово из списка служебных слов, написанных заглавными
                elif word.upper() in PRONOUNS_OR_FUNCTION_WORDS and word.isupper():
                     current_sentence_words.append(word.lower()) # Приводим к нижнему регистру
                     is_first_word_of_sentence = False
                elif is_first_word_of_sentence and word and word[0].isupper():
                     # Сохраняем заглавную букву в начале первого слова предложения
                     current_sentence_words.append(word[0] + word[1:].lower())
                     is_first_word_of_sentence = False
                else:
                    current_sentence_words.append(word.lower()) # Приводим остальные слова к нижнему регистру
    # Добавляем последнее предложение, если оно не было завершено знаком препинания
    if current_sentence_words:
         processed_sentences.append(" ".join(current_sentence_words))

    text = " ".join(processed_sentences).strip() # Объединяем части обратно

    # Удаление повторяющихся подряд слов (e.g., "при при поступлении")
    text = re.sub(r'\b(\w+)\s+\1\b', r'\1', text)

    # Ищем последовательность из слов, за которой следует та же последовательность
    match_repeats = True
    while match_repeats:
        pattern = r'\b((?:\w+\s+){1,}\w+)\s+\1\b' # Ищем повторение
        new_text = re.sub(pattern, r'\1', text, flags=re.IGNORECASE) # Игнорируем регистр при поиске повторов
        if new_text == text:
            match_repeats = False # Повторов больше нет
        text = new_text



    # финальная очистка
    text = re.sub(r'(.)(?!\1)(?<![\.,])\1+', r'\1', text)

    # Удаление множественных пробелов
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Применение очистки к столбцу text_features
if not df_results.empty:
    print("Применяю дополнительную очистку к столбцу 'text_features'...")
    df_results['text_features'] = df_results['text_features'].apply(further_clean_text)
    print("Очистка завершена. Первые 5 строк с новым столбцом 'text_features':")
    display(df_results[['text_features']].head())
else:
    print("DataFrame пуст, дополнительная очистка не выполнена.")

## MedGemma

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Доступно {torch.cuda.device_count()} GPU.")
    print(f"Текущее GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU недоступно, используется CPU.")

# Для воспроизводимости
SEED = 17

In [ ]:
# Отбираем только данные с известными таргетами
df_labeled = df_results[df_results['mace_target'].notna()].copy()
df_labeled['mace_target'] = df_labeled['mace_target'].astype(int)
print(f"\nРазмер датасета для обучения и теста: {len(df_labeled)}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch

MODEL_NAME = "google/medgemma-4b-it"

# Квантизация для уменьшения использования памяти
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16, # Для A100 используем bfloat16
    bnb_4bit_use_double_quant=True,
)

# Загружаем базовую модель
model = Gemma3ForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="eager"
)
MODEL_MAX_SEQ_LENGTH = 2048
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    model_max_length=MODEL_MAX_SEQ_LENGTH,
    padding_side="right",  # Явно задаем сторону паддинга, "right" - частый выбор
    truncation_side="right" # Явно задаем сторону обрезки
)


num_labels = 2
model.config.num_labels = num_labels
model.config.id2label = {i: f'LABEL_{i}' for i in range(num_labels)}
model.config.label2id = {f'LABEL_{i}': i for i in range(num_labels)}


# Добавляем классификационную голову поверх базовой модели
classifier_input_dim = model.config.hidden_size


# Настройка padding token если он не установлен
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print(f"\nТокенизатор загружен. Pad token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")
print(f"Базовая модель {MODEL_NAME} загружена с добавленной классификационной головой.")
print(f"Конфигурация модели (первые несколько параметров):")
for k, v in list(model.config.to_dict().items()):
    print(f"  {k}: {v}")

In [ ]:
# Проверим model_max_length токенизатора
CONTEXT_WINDOW_SIZE = tokenizer.model_max_length
print(f"\nМаксимальная длина последовательности для токенизатора (tokenizer.model_max_length): {CONTEXT_WINDOW_SIZE}")
# Устанавливаем PRACTICAL_MAX_LENGTH согласно рекомендациям для Med-Gemini 4b
PRACTICAL_MAX_LENGTH = MODEL_MAX_SEQ_LENGTH
print(f"Практическая максимальная длина для токенизации: {PRACTICAL_MAX_LENGTH}")

In [ ]:
token_lengths = []
for text in df_labeled['text_features']:
    tokens = tokenizer.encode(text, add_special_tokens=True)
    token_lengths.append(len(tokens))

max_observed_len = np.max(token_lengths)
avg_observed_len = np.mean(token_lengths)
median_observed_len = np.median(token_lengths)
percentile_95_len = np.percentile(token_lengths, 95)

print(f"Анализ длин токенизированных текстов (на {len(df_labeled)} примерах):")
print(f"  Максимальная наблюдаемая длина: {max_observed_len} токенов")
print(f"  Средняя наблюдаемая длина: {avg_observed_len:.2f} токенов")
print(f"  Медианная наблюдаемая длина: {median_observed_len:.2f} токенов")
print(f"  95-й перцентиль длины: {percentile_95_len:.2f} токенов")

if max_observed_len > PRACTICAL_MAX_LENGTH:
    print(f"\n Максимальная наблюдаемая длина ({max_observed_len}) превышает выбранную практическую максимальную длину ({PRACTICAL_MAX_LENGTH}).")
    num_truncated = sum(1 for length in token_lengths if length > PRACTICAL_MAX_LENGTH)
    print(f"   Примерно {num_truncated} из {len(df_labeled)} ({num_truncated/len(df_labeled)*100:.2f}%) текстов будут проанализированны со скользящим окном.")
else:
    print(f"\n Все тексты ({max_observed_len} токенов макс.) помещаются в выбранную практическую максимальную длину ({PRACTICAL_MAX_LENGTH}).")

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_labeled['text_features'],
    df_labeled['mace_target'],
    test_size=0.20, # 20% на тест
    random_state=SEED,
    stratify=df_labeled['mace_target']
)

print(f"Размер обучающей выборки: {len(train_texts)}")
print(f"Размер тестовой выборки: {len(test_texts)}")
print(f"Распределение классов в обучающей выборке: {pd.Series(train_labels).value_counts(normalize=True)}")
print(f"Распределение классов в тестовой выборке: {pd.Series(test_labels).value_counts(normalize=True)}")

# Создание словарей для Dataset
train_data = {'text': list(train_texts), 'label': list(train_labels)}
test_data = {'text': list(test_texts), 'label': list(test_labels)}

# Преобразование в Hugging Face Dataset
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

raw_datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print("\nСтруктура датасета Hugging Face:")
print(raw_datasets)

# Функция токенизации
def tokenize_function(examples):
    # Используем PRACTICAL_MAX_LENGTH, определенную ранее
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=PRACTICAL_MAX_LENGTH)
    # padding=False здесь, т.к. DataCollatorWithPadding сделает это динамически для каждого батча

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=["text"])

# Data collator для динамического паддинга батчей до максимальной длины в батче
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("\nПример токенизированного датасета:")
print(tokenized_datasets['train'][0])

In [ ]:
from peft import (LoraConfig,
                  PeftModel,
                  prepare_model_for_kbit_training,
                  get_peft_model,
                  PeftModelForSequenceClassification,
                  PeftConfig)
from transformers.modeling_outputs import SequenceClassifierOutput

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM # Убедитесь, что Gemma3ForCausalLM импортирован
from peft import PeftModelForSequenceClassification, PeftConfig
from transformers.modeling_outputs import SequenceClassifierOutput
import torch
import torch.nn as nn

class Gemma3ForSequenceClassification(PeftModelForSequenceClassification):
    def __init__(self, peft_config: PeftConfig, model: AutoModelForCausalLM, adapter_name="default"):
        super().__init__(model, peft_config, adapter_name)
        self.num_labels = 2
        self.problem_type = "single_label_classification" # Используем "single_label_classification" для CrossEntropyLoss с метками [0, 1]

        # Добавление классификационной головы к базовой модели, если ее нет
        # self.base_model.model - это оригинальная Gemma3ForCausalLM
        if not hasattr(self.base_model.model, 'score'):
            classifier_input_dim = self.base_model.model.config.hidden_size
            self.score_head = nn.Linear(classifier_input_dim, self.num_labels)

            # Перемещаем новую голову на то же устройство, что и базовая модель
            try:
                # Пытаемся определить устройство модели более надежным способом
                device = next(self.base_model.model.parameters()).device
                self.score_head.to(device)
            except StopIteration: # Если у модели нет параметров
                # Фолбэк или логирование, если устройство не определено
                print("Warning: Could not determine model device for score_head. Defaulting to cpu.")
                self.score_head.to('cpu')


        elif hasattr(self.base_model.model, 'score') and isinstance(self.base_model.model.score, nn.Linear):
            # Если score уже существует на базовой модели
            # и это линейный слой, используем его.
            self.score_head = self.base_model.model.score
        else:
            # Если score есть, но это не nn.Linear, или другая непредвиденная ситуация
            raise TypeError(f"Expected self.base_model.model.score to be nn.Linear or not exist, but found {type(self.base_model.model.score)}")

        # Убедимся, что конфигурация модели отражает количество меток
        if self.config.num_labels != self.num_labels:
            self.config.num_labels = self.num_labels
        if self.config.problem_type != self.problem_type:
            self.config.problem_type = self.problem_type


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,     # Параметр из сигнатуры, указывает, хочет ли Trainer/вызывающий вернуть attentions
        output_hidden_states=None,  # Параметр из сигнатуры, указывает, хочет ли Trainer/вызывающий вернуть ВСЕ hidden_states
        return_dict=None,
        **kwargs):

        # Определяем, нужно ли возвращать словарь, основываясь на аргументе или конфигурации
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        # Для внутреннего вызова базовой модели лучше всегда использовать return_dict=True для консистентности
        effective_return_dict_for_base_call = True

        # Вызов базовой PEFT-модели (которая оборачивает Gemma3ForCausalLM)
        # Передаем запросы на output_attentions/output_hidden_states от вызывающей стороны
        # к базовой модели. Таким образом, если Trainer их запрашивает, они будут вычислены.
        # Мы всегда запрашиваем output_hidden_states=True для базовой модели,
        # так как нам нужен последний скрытый слой для пулинга.
        # Gradient checkpointing должен обрабатывать память для активаций промежуточных слоев базовой модели.
        outputs_from_base = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=True, # Нам нужен доступ к outputs_from_base.hidden_states
            return_dict=effective_return_dict_for_base_call,
            **kwargs
        )

        # Извлекаем скрытые состояния последнего слоя.
        # outputs_from_base.hidden_states - это кортеж всех скрытых состояний слоев.
        last_layer_token_hidden_states = outputs_from_base.hidden_states[-1]

        # Пулинг: выбираем скрытое состояние последнего значащего токена для каждой последовательности.
        if attention_mask is not None:
            sequence_lengths = torch.sum(attention_mask, dim=1) - 1
            sequence_lengths = torch.clamp(sequence_lengths, min=0) # Для обработки пустых последовательностей

            pooled_output = last_layer_token_hidden_states[
                torch.arange(last_layer_token_hidden_states.size(0), device=last_layer_token_hidden_states.device),
                sequence_lengths
            ]
        else:
            # Если нет attention_mask, берем последний токен (менее надежно для паддинга)
            pooled_output = last_layer_token_hidden_states[:, -1, :]

        # Получаем логиты из нашей классификационной головы
        # Если голова была добавлена как self.base_model.model.score:
        # logits = self.base_model.model.score(pooled_output)
        # Если голова определена как self.score_head в этом классе:
        logits = self.score_head(pooled_output)


        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            # Убедимся, что labels имеют тип LongTensor, как ожидает CrossEntropyLoss
            loss = loss_fct(logits.view(-1, self.num_labels), labels.long())

        # Управляем возвращаемыми значениями в зависимости от return_dict,
        if not return_dict: # Если вызывающий явно просил кортеж
            output_items = (logits,)
            # Включаем полные hidden_states или attentions только если они были запрошены
            # через аргументы output_hidden_states/output_attentions этого метода forward
            if output_hidden_states and hasattr(outputs_from_base, 'hidden_states') and outputs_from_base.hidden_states is not None:
                output_items += (outputs_from_base.hidden_states,)
            if output_attentions and hasattr(outputs_from_base, 'attentions') and outputs_from_base.attentions is not None:
                output_items += (outputs_from_base.attentions,)

            return ((loss,) + output_items) if loss is not None else output_items

        # Если return_dict=True
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            # Возвращаем полные hidden_states/attentions только если вызывающий их просил.
            # В противном случае возвращаем None, чтобы Trainer не накапливал их.
            hidden_states=outputs_from_base.hidden_states if output_hidden_states else None,
            attentions=outputs_from_base.attentions if output_attentions else None,
        )

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

peft_model = Gemma3ForSequenceClassification(lora_config, model)

print("\nМодель после применения LORA:")
peft_model.print_trainable_parameters()

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    try:
        auc = roc_auc_score(labels, logits[:,1]) # Вероятности для положительного класса
    except ValueError:
        auc = 0.5
    return {"accuracy": acc, "f1": f1, "auc": auc}

### Train only with targets

In [ ]:
OUTPUT_DIR = "./results_medgemma_lora_mace"
LOGGING_DIR = './logs_medgemma_lora_mace'

training_args = TrainingArguments(
    label_names=["labels"],
    output_dir=OUTPUT_DIR,
    num_train_epochs=15,
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    eval_strategy="epoch", # Оценивать после каждой эпохи
    save_strategy="epoch",       # Сохранять модель после каждой эпохи
    load_best_model_at_end=True, # Загрузить лучшую модель в конце обучения
    metric_for_best_model="auc",  # Метрика для определения лучшей модели
    logging_dir=LOGGING_DIR,
    logging_steps=1,
    warmup_steps=3,
    report_to="tensorboard",
    fp16=False, # Для A100 используем bf16
    bf16=True,  # Включаем bfloat16 для A100.
    gradient_checkpointing=True, # Включаем градиентный чекпоинтинг для экономии памяти
    max_grad_norm=0.3, # Добавляем клиппинг
    seed=SEED,
)

# Создание Trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

print("Отображение Warnings отключено.")

In [ ]:
print("Начинаем обучение модели...")
train_result = trainer.train()

# Сохранение метрик обучения
train_metrics = train_result.metrics
trainer.log_metrics("train", train_metrics)
trainer.save_metrics("train", train_metrics)

print("\nОбучение завершено.")

In [ ]:
print("\nОценка лучшей модели на тестовой выборке:")
eval_metrics = trainer.evaluate()

trainer.log_metrics("eval", eval_metrics)
trainer.save_metrics("eval", eval_metrics)

print("\nМетрики на тестовой выборке:")
for key, value in eval_metrics.items():
    print(f"  {key}: {value:.4f}")

In [ ]:
ADAPTER_OUTPUT_DIR = "/content/drive/My Drive/medgemma_lora_mace_adapter"
peft_model.save_pretrained(ADAPTER_OUTPUT_DIR)
tokenizer.save_pretrained(ADAPTER_OUTPUT_DIR) # Сохраняем токенизатор вместе с адаптером для удобства

print(f"\nОбученный LoRA адаптер и токенизатор сохранены в: {ADAPTER_OUTPUT_DIR}")

### Inference on the full dataset

In [ ]:
train_id, test_id, train_labels, test_labels = train_test_split(
    df_labeled['patient_id'],
    df_labeled['mace_target'],
    test_size=0.20, # 20% на тест
    random_state=SEED,
    stratify=df_labeled['mace_target']
)

train_data_labels = {'text': list(train_texts), 'label': list(train_labels), 'patient_id': train_id}
test_data_labels = {'text': list(test_texts), 'label': list(test_labels), 'patient_id': test_id}

In [ ]:
if 'trainer' in locals() and hasattr(trainer, 'model'):
    trainer.model.eval()
elif 'peft_model' in locals(): # Если trainer был удален, но peft_model осталась
    peft_model.eval()
else:
    raise ValueError("Модель (trainer или peft_model) не найдена. Убедитесь, что обучение было завершено.")

# --- Предсказания для обучающей выборки ---
print("\nПолучение предсказаний для обучающей выборки...")
if "train" in tokenized_datasets:
    train_pred_output = trainer.predict(tokenized_datasets["train"])
    train_logits = train_pred_output.predictions
    # Преобразование логитов в вероятности для класса 1 (MACE)
    train_probabilities_mace = torch.softmax(torch.tensor(train_logits), dim=-1)[:, 1].numpy()
    # Исходные метки и ID пациентов из датасета
    train_actual_labels = tokenized_datasets["train"]["label"]
    train_patient_ids_ds = train_data_labels["patient_id"]

    train_results_list = []
    for i in range(len(train_patient_ids_ds)):
        train_results_list.append({
            'patient_id': train_patient_ids_ds.values[i],
            'dataset_type': 'train',
            'mace_probability_score': train_probabilities_mace[i],
            'actual_mace_target': train_actual_labels[i]
        })
    print(f"Обработано {len(train_results_list)} записей из обучающей выборки.")
else:
    print("Обучающая выборка (tokenized_datasets['train']) не найдена.")
    train_results_list = []

# --- Предсказания для тестовой выборки ---
print("\nПолучение предсказаний для тестовой выборки...")
if "test" in tokenized_datasets:
    test_pred_output = trainer.predict(tokenized_datasets["test"])
    test_logits = test_pred_output.predictions
    # Преобразование логитов в вероятности для класса 1 (MACE)
    test_probabilities_mace = torch.softmax(torch.tensor(test_logits), dim=-1)[:, 1].numpy()
    # Исходные метки и ID пациентов из датасета
    test_actual_labels = tokenized_datasets["test"]["label"]
    test_patient_ids_ds = test_data_labels["patient_id"]

    test_results_list = []
    for i in range(len(test_patient_ids_ds)):
        test_results_list.append({
            'patient_id': test_patient_ids_ds.values[i],
            'dataset_type': 'test',
            'mace_probability_score': test_probabilities_mace[i],
            'actual_mace_target': test_actual_labels[i]
        })
    print(f"Обработано {len(test_results_list)} записей из тестовой выборки.")
else:
    print("Тестовая выборка (tokenized_datasets['test']) не найдена.")
    test_results_list = []

# --- Объединение результатов в один DataFrame ---
if train_results_list or test_results_list:
    final_scores_df = pd.DataFrame(train_results_list + test_results_list)
    print("\nИтоговый DataFrame с предсказаниями (первые 5 строк):")
    display(final_scores_df.head())
    print(f"\nРазмер итогового DataFrame: {final_scores_df.shape}")

    # --- Сохранение DataFrame на Google Диск ---
    output_scores_filename = "/content/drive/My Drive/gemma_mace_predictions.parquet"
    try:
        final_scores_df.to_parquet(output_scores_filename, index=False)
        print(f"\nDataFrame с предсказаниями успешно сохранен в: {output_scores_filename}")
    except Exception as e:
        print(f"\nОшибка при сохранении DataFrame: {e}")
else:
    print("\nНет данных для создания итогового DataFrame.")

# Очистка памяти
import gc
del train_pred_output, test_pred_output, train_logits, test_logits
del train_probabilities_mace, test_probabilities_mace
del final_scores_df
gc.collect()
torch.cuda.empty_cache()